# Environment and Dependencies

In [ ]:
!pip install transformers
!pip install scipy

In [ ]:
!pip install soundfile

In [ ]:
import numpy as np

# Musicgen small model

In [ ]:
from transformers import AutoProcessor, MusicgenForConditionalGeneration

processor = AutoProcessor.from_pretrained("facebook/musicgen-small")
model = MusicgenForConditionalGeneration.from_pretrained("facebook/musicgen-small")



# Music generation from text prompt

In [ ]:
# Text prompt
text=["lo-fi music using indian classical music instruments and a rock song played on it"]

In [ ]:
# Text prompt preprocessing and inference

inputs = processor(
    text=text,
    padding=True,
    return_tensors="pt",
)

audio_values = model.generate(**inputs, max_new_tokens=256)

In [ ]:
# Run this cell to listen to music generated
from IPython.display import Audio

sampling_rate = model.config.audio_encoder.sampling_rate
Audio(audio_values[0].cpu().numpy(), rate=sampling_rate)

In [ ]:
# Run this cell to save music file
import scipy

sampling_rate = model.config.audio_encoder.sampling_rate
scipy.io.wavfile.write("musicgen_out.wav", rate=sampling_rate, data=audio_values[0, 0].numpy())

# Music generation from Text and audio prompt

In [ ]:
import soundfile as sf

stereo_path = "/content/rosamunde_overture_excerpt_2_alt_proud_music_preview.wav"

data, sampling_rate = sf.read(stereo_path)

# Model takes only mono audio i.e only single channel
if len(data.shape) > 1:

  mono_data = np.mean(data, axis=1)
else:

  mono_data = data




text_prompt = ["Play the music using indian classical instruments like sitar"]

inputs = processor(
    audio=mono_data,
    sampling_rate = model.config.audio_encoder.sampling_rate,
    text=text_prompt,
    padding=True,
    return_tensors="pt",
)

audio_values = model.generate(**inputs, do_sample=True, guidance_scale=3, max_new_tokens=256)


Audio(audio_values[0].cpu().numpy(), rate=sampling_rate)
